# This specific notebook is outdated as of 10/21/2025.
## TODO: Redesign notebook to use new optimizer framework

In [ ]:
import datetime as dt
import polars as pl
from trading_engine.core import (
    read_data, create_model_state, orchestrate_model_backtests,
    orchestrate_model_simulations, orchestrate_portfolio_aggregation,
    orchestrate_portfolio_optimizations, orchestrate_portfolio_simulations
)
from trading_engine.models import MODELS
from trading_engine.optimizers import OPTIMIZERS

from trading_engine.model_state.registry import momentum
from common.constants import ProcessingMode

import datetime

In [ ]:
# Define your aggregator (this is actually an aggregator, not an optimizer!)

import datetime as _dt
from typing import Callable, Dict, List

import polars as pl
from polars import LazyFrame

_EPS = 1e-9


def MinAvgDrawdownAggregator(window_days: int = 90) -> Callable[[Dict[str, LazyFrame], Dict], LazyFrame]:
    """
    Build a portfolio by weighting model insights inversely to their average drawdown
    over the last `window_days`. Models with lower recent drawdown get higher weight.
    """

    def _avg_drawdown_recent(metrics_df: pl.DataFrame) -> float:
        if metrics_df is None or metrics_df.is_empty():
            return float("inf")

        # Standardize date to pl.Date
        dt = metrics_df.get_column("date")
        if dt.dtype == pl.Utf8:
            metrics_df = metrics_df.with_columns(
                pl.col("date").str.strptime(pl.Date, strict=False).alias("date")
            )
        elif dt.dtype == pl.Datetime:
            metrics_df = metrics_df.with_columns(pl.col("date").dt.date().alias("date"))
        # If already pl.Date, leave as-is.

        # Window: last `window_days` calendar days from max date
        max_d = metrics_df.select(pl.col("date").max()).item()
        if max_d is None:
            return float("inf")
        start = max_d - _dt.timedelta(days=window_days)

        # Drawdown magnitude (use abs in case stored negative)
        win = metrics_df.filter(pl.col("date") >= pl.lit(start))
        if win.is_empty():
            win = metrics_df  # fallback to full history

        val = win.select(pl.col("drawdown").abs().mean()).item()
        if val is None or not (val == val):  # NaN check
            return float("inf")
        return float(val)

    def _coef_from_avg_dd(avg_dd: float) -> float:
        # Lower avg drawdown => higher coefficient. Guard with epsilon.
        if avg_dd == float("inf"):
            return 0.0
        return 1.0 / (_EPS + max(avg_dd, 0.0))

    def run(model_insights: Dict[str, LazyFrame], backtest_results: Dict) -> LazyFrame:
        if not model_insights:
            return pl.DataFrame({"date": []}).lazy()

        # 1) Compute per-model coefficients from backtest_metrics
        coefs: Dict[str, float] = {}
        for mname, lf in model_insights.items():
            metrics = None
            if mname in backtest_results:
                # New structure: backtest_results[mname] = {"full_backtest_results": {...}, "backtest_results": {...}}
                # Use full_backtest_results for aggregators (has full history)
                full_results = backtest_results[mname].get("full_backtest_results", {})
                metrics = full_results.get("backtest_results")

            # Accept either Polars or Pandas metrics (convert if needed)
            if metrics is None:
                avg_dd = float("inf")
            elif isinstance(metrics, pl.DataFrame):
                avg_dd = _avg_drawdown_recent(metrics)
            else:
                try:
                    import pandas as pd  # optional dependency
                    if isinstance(metrics, pd.DataFrame):
                        avg_dd = _avg_drawdown_recent(pl.from_pandas(metrics))
                    else:
                        avg_dd = float("inf")
                except Exception:
                    avg_dd = float("inf")

            coefs[mname] = _coef_from_avg_dd(avg_dd)

        # Normalize coefficients to sum to 1; if all zero, fall back to equal weight
        total = sum(coefs.values())
        if total <= _EPS:
            n = float(len(model_insights))
            coefs = {k: 1.0 / n for k in model_insights.keys()}
        else:
            coefs = {k: v / total for k, v in coefs.items()}

        # 2) Scale each model's weights by its coefficient and combine
        longs: List[LazyFrame] = []
        for mname, lf in model_insights.items():
            coef = coefs.get(mname, 0.0)

            # Multiply all non-'date' columns by coef (robust across Polars versions)
            names = lf.collect_schema().names()
            wcols = [c for c in names if c != "date"]
            if not wcols:
                continue

            scaled = lf.with_columns([(pl.col(c) * pl.lit(coef)).alias(c) for c in wcols])

            # Use unpivot instead of melt (updated API)
            long = scaled.unpivot(index="date", variable_name="ticker", value_name="w")
            longs.append(long)

        if not longs:
            return pl.DataFrame({"date": []}).lazy()

        # 3) Sum across models in long space, pivot to wide
        combined_long_lf = pl.concat(longs, how="vertical").group_by(["date", "ticker"]).agg(
            pl.col("w").sum().alias("w")
        )

        combined_wide_df = (
            combined_long_lf
            .collect(engine="streaming")  # pivot is a DataFrame op in many Polars versions
            .pivot(values="w", index="date", on="ticker", aggregate_function="first")
            .sort("date")
        )
        return combined_wide_df.lazy()

    return run


In [ ]:
aggregator_registry = {
    "min_avg_drawdown": {
        "function": MinAvgDrawdownAggregator(window_days=252),
        "lookback": 0,  # No additional lookback needed
    }
}

In [ ]:
# 1) experiment config
universe = [
  "SPY-US", "SLV-US", "GLD-US", "TLT-US", "USO-US", "UNG-US", "IXJ-US",
  "KXI-US", "JXI-US", "IXG-US", "IXN-US", "RXI-US", "MXI-US", "EXI-US",
  "IXC-US", "IEI-US", "SHY-US", "BIL-US", "JPXN-US", "INDA-US", "MCHI-US",
  "EZU-US", "IBIT-US", "ETHA-US", "VIXY-US"
]
features = ["close_momentum_10"]                   # must exist in FEATURES
models   = ["RXI_TLT_pml_10", "GLD_USO_nml_10"]    # keys in MODELS
aggregators = ["min_avg_drawdown"]                 # keys in aggregator_registry (custom)
optimizers   = ["mean_variance_constrained"]       # keys in OPTIMIZERS (optional final stage)
initial_value = 1_000_000
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2025, 1, 1)

In [ ]:
# 2) build model state + prices (cached locally)
raw_data_bundle = read_data(include_supplemental=True)
model_state_bundle, prices = create_model_state(
    raw_data_bundle=raw_data_bundle,
    features=features,
    start_date=start_date,
    end_date=end_date,
    universe=universe
)

In [ ]:
# 3) run model backtests + simulations
model_insights = orchestrate_model_backtests(
    model_state_bundle=model_state_bundle,
    models=models,
    universe=universe,
    registry=MODELS  # pass in your custom models registry instead of pulling default prod registry
)

model_simulations = orchestrate_model_simulations(
    prices=prices,
    model_insights=model_insights,
    start_date=start_date,
    end_date=end_date
)

In [ ]:
# 4) aggregate + optimize portfolio and simulate
aggregated_insights = orchestrate_portfolio_aggregation(
    model_insights=model_insights,
    backtest_results=model_simulations,
    universe=universe,
    aggregators=aggregators,
    start_date=start_date,
    end_date=end_date,
    registry=aggregator_registry,  # Use custom aggregator registry
)

optimizer_insights = orchestrate_portfolio_optimizations(
    prices=prices,
    aggregated_insights=aggregated_insights,
    universe=universe,
    optimizers=optimizers,
)

optimizer_simulations = orchestrate_portfolio_simulations(
    prices=prices,
    portfolio_insights=optimizer_insights,
    start_date=start_date,
    end_date=end_date,
    initial_value=initial_value,
)

In [ ]:
# 5) visualize one result (example: mean_variance_constrained)
optimizer_simulations["mean_variance_constrained"]["backtest_metrics"]